## रिग्रेशन का परिचय - पाठ 1

#### इसे परिप्रेक्ष्य में रखना

✅ रिग्रेशन के कई प्रकार के तरीके होते हैं, और आप कौन सा तरीका चुनते हैं, यह इस पर निर्भर करता है कि आप किस प्रश्न का उत्तर ढूंढ रहे हैं। यदि आप किसी दिए गए उम्र के व्यक्ति की संभावित ऊंचाई का अनुमान लगाना चाहते हैं, तो आप `linear regression` का उपयोग करेंगे, क्योंकि आप एक **संख्यात्मक मान** की तलाश में हैं। यदि आप यह जानना चाहते हैं कि किसी प्रकार के व्यंजन को शाकाहारी माना जाना चाहिए या नहीं, तो आप एक **श्रेणी निर्धारण** की तलाश में हैं, इसलिए आप `logistic regression` का उपयोग करेंगे। आप बाद में लॉजिस्टिक रिग्रेशन के बारे में और जानेंगे। डेटा से पूछे जा सकने वाले कुछ प्रश्नों के बारे में सोचें, और इनमें से कौन सा तरीका अधिक उपयुक्त होगा।

इस खंड में, आप [डायबिटीज़ पर एक छोटे डेटा सेट](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) के साथ काम करेंगे। कल्पना करें कि आप मधुमेह रोगियों के लिए एक उपचार का परीक्षण करना चाहते हैं। मशीन लर्निंग मॉडल यह निर्धारित करने में मदद कर सकते हैं कि कौन से रोगी उपचार के लिए बेहतर प्रतिक्रिया देंगे, विभिन्न चर के संयोजनों के आधार पर। यहां तक कि एक बहुत ही बुनियादी रिग्रेशन मॉडल, जब विज़ुअलाइज़ किया जाता है, तो उन चर के बारे में जानकारी दिखा सकता है जो आपके सैद्धांतिक नैदानिक परीक्षणों को व्यवस्थित करने में मदद कर सकते हैं।

तो चलिए, इस कार्य को शुरू करते हैं!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>@allison_horst द्वारा कलाकृति</figcaption>

<!--![@allison_horst द्वारा कलाकृति](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.hi.jpg)<br>@allison_horst द्वारा कलाकृति-->


## 1. हमारे टूल सेट को लोड करना

इस कार्य के लिए, हमें निम्नलिखित पैकेजों की आवश्यकता होगी:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) एक [R पैकेजों का संग्रह](https://www.tidyverse.org/packages) है, जिसे डेटा साइंस को तेज़, आसान और मज़ेदार बनाने के लिए डिज़ाइन किया गया है!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क एक [पैकेजों का संग्रह](https://www.tidymodels.org/packages/) है, जो मॉडलिंग और मशीन लर्निंग के लिए उपयोगी है।

आप इन्हें इस प्रकार इंस्टॉल कर सकते हैं:

`install.packages(c("tidyverse", "tidymodels"))`

नीचे दिया गया स्क्रिप्ट यह जांचता है कि इस मॉड्यूल को पूरा करने के लिए आवश्यक पैकेज आपके पास हैं या नहीं, और यदि कुछ पैकेज गायब हैं तो उन्हें आपके लिए इंस्टॉल कर देता है।


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



अब, आइए इन शानदार पैकेजों को लोड करें और उन्हें हमारे वर्तमान R सत्र में उपलब्ध कराएं। (यह केवल उदाहरण के लिए है, `pacman::p_load()` ने यह पहले ही आपके लिए कर दिया है)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. डायबिटीज डेटा सेट

इस अभ्यास में, हम अपने रिग्रेशन कौशल का प्रदर्शन करेंगे और डायबिटीज डेटा सेट पर भविष्यवाणियां करेंगे। [डायबिटीज डेटा सेट](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) में डायबिटीज से संबंधित `442 नमूनों` का डेटा शामिल है, जिसमें 10 भविष्यवाणी करने वाले फीचर वेरिएबल्स हैं: `उम्र`, `लिंग`, `बॉडी मास इंडेक्स`, `औसत ब्लड प्रेशर`, और `छह ब्लड सीरम माप`, साथ ही एक परिणाम वेरिएबल `y`: एक साल बाद बीमारी की प्रगति का मात्रात्मक माप।

|अवलोकनों की संख्या|442|
|------------------|:---|
|भविष्यवाणी करने वाले वेरिएबल्स की संख्या|पहले 10 कॉलम संख्यात्मक भविष्यवाणी करने वाले हैं|
|परिणाम/लक्ष्य|कॉलम 11 एक साल बाद बीमारी की प्रगति का मात्रात्मक माप है|
|भविष्यवाणी करने वाले वेरिएबल्स की जानकारी|- उम्र (सालों में)
||- लिंग
||- बॉडी मास इंडेक्स (BMI)
||- औसत ब्लड प्रेशर (BP)
||- s1 tc, कुल सीरम कोलेस्ट्रॉल
||- s2 ldl, लो-डेंसिटी लिपोप्रोटीन
||- s3 hdl, हाई-डेंसिटी लिपोप्रोटीन
||- s4 tch, कुल कोलेस्ट्रॉल / HDL
||- s5 ltg, संभवतः सीरम ट्राइग्लिसराइड्स स्तर का लॉग
||- s6 glu, ब्लड शुगर स्तर|

> 🎓 याद रखें, यह सुपरवाइज्ड लर्निंग है, और हमें एक नामित 'y' लक्ष्य की आवश्यकता है।

R के साथ डेटा को मैनिपुलेट करने से पहले, आपको डेटा को R की मेमोरी में इंपोर्ट करना होगा, या डेटा तक पहुंचने के लिए R के साथ एक कनेक्शन बनाना होगा।

> [readr](https://readr.tidyverse.org/) पैकेज, जो Tidyverse का हिस्सा है, R में आयताकार डेटा को तेज़ और आसान तरीके से पढ़ने की सुविधा प्रदान करता है।

अब, आइए इस स्रोत URL से प्रदान किए गए डायबिटीज डेटा सेट को लोड करें: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

साथ ही, हम अपने डेटा पर एक त्वरित जांच करेंगे `glimpse()` का उपयोग करके और पहले 5 पंक्तियों को `slice()` के माध्यम से प्रदर्शित करेंगे।

आगे बढ़ने से पहले, आइए कुछ ऐसा भी जानें जिसे आप अक्सर R कोड में देखेंगे 🥁🥁: पाइप ऑपरेटर `%>%`

पाइप ऑपरेटर (`%>%`) तार्किक क्रम में ऑपरेशन्स को अंजाम देता है, जिसमें एक ऑब्जेक्ट को आगे किसी फंक्शन या कॉल एक्सप्रेशन में पास किया जाता है। आप पाइप ऑपरेटर को अपने कोड में "और फिर" कहने जैसा समझ सकते हैं।


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` हमें दिखाता है कि इस डेटा में 442 पंक्तियाँ और 11 कॉलम हैं, और सभी कॉलम का डेटा प्रकार `double` है।

<br>

> `glimpse()` और `slice()` [`dplyr`](https://dplyr.tidyverse.org/) की फंक्शन हैं। Dplyr, जो Tidyverse का हिस्सा है, डेटा को मैनेज करने का एक व्याकरण है, जो एक समान सेट के क्रियाओं (verbs) को प्रदान करता है, जिससे आप डेटा मैनेजमेंट से जुड़ी आम समस्याओं को हल कर सकते हैं।

<br>

अब जब हमारे पास डेटा है, तो इस अभ्यास के लिए हम एक फीचर (`bmi`) पर ध्यान केंद्रित करेंगे। इसके लिए हमें वांछित कॉलम को चुनना होगा। तो, हम यह कैसे करेंगे?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) हमें डेटा फ्रेम में कॉलम को *चुनने* (और वैकल्पिक रूप से उनका नाम बदलने) की अनुमति देता है।


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. प्रशिक्षण और परीक्षण डेटा

सुपरवाइज्ड लर्निंग में डेटा को दो उपसेट्स में *विभाजित* करना आम प्रथा है; एक (आमतौर पर बड़ा) सेट जिसका उपयोग मॉडल को प्रशिक्षित करने के लिए किया जाता है, और एक छोटा "होल्ड-बैक" सेट जिसका उपयोग यह देखने के लिए किया जाता है कि मॉडल ने कैसा प्रदर्शन किया।

अब जब हमारे पास डेटा तैयार है, तो हम देख सकते हैं कि क्या मशीन इस डेटासेट में संख्याओं के बीच एक तार्किक विभाजन निर्धारित करने में मदद कर सकती है। हम [rsample](https://tidymodels.github.io/rsample/) पैकेज का उपयोग कर सकते हैं, जो Tidymodels फ्रेमवर्क का हिस्सा है, एक ऑब्जेक्ट बनाने के लिए जिसमें डेटा को *कैसे* विभाजित करना है इसकी जानकारी होती है, और फिर दो और rsample फ़ंक्शन्स का उपयोग करके बनाए गए प्रशिक्षण और परीक्षण सेट्स को निकाल सकते हैं:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Tidymodels के साथ एक लीनियर रिग्रेशन मॉडल ट्रेन करें

अब हम अपने मॉडल को ट्रेन करने के लिए तैयार हैं!

Tidymodels में, आप `parsnip()` का उपयोग करके तीन मुख्य अवधारणाओं को निर्दिष्ट करके मॉडल सेट करते हैं:

-   मॉडल का **प्रकार** विभिन्न मॉडलों को अलग करता है, जैसे लीनियर रिग्रेशन, लॉजिस्टिक रिग्रेशन, डिसीजन ट्री मॉडल, आदि।

-   मॉडल का **मोड** सामान्य विकल्पों जैसे रिग्रेशन और क्लासिफिकेशन को शामिल करता है; कुछ मॉडल प्रकार इनमें से किसी एक को सपोर्ट करते हैं, जबकि कुछ केवल एक ही मोड रखते हैं।

-   मॉडल का **इंजन** वह गणनात्मक टूल है जिसका उपयोग मॉडल को फिट करने के लिए किया जाएगा। अक्सर ये R पैकेज होते हैं, जैसे **`"lm"`** या **`"ranger"`**।

यह मॉडलिंग जानकारी एक मॉडल स्पेसिफिकेशन में कैप्चर की जाती है, तो चलिए इसे बनाते हैं!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

एक बार मॉडल को *निर्धारित* कर लिया गया हो, तो मॉडल को [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) फ़ंक्शन का उपयोग करके `अनुमानित` या `प्रशिक्षित` किया जा सकता है, आमतौर पर एक सूत्र और कुछ डेटा का उपयोग करके।

`y ~ .` का मतलब है कि हम `y` को पूर्वानुमानित मात्रा/लक्ष्य के रूप में फिट करेंगे, जिसे सभी पूर्वानुमानकों/विशेषताओं द्वारा समझाया जाएगा, यानी `.` (इस मामले में, हमारे पास केवल एक पूर्वानुमानक है: `bmi`)।


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

मॉडल आउटपुट से, हम प्रशिक्षण के दौरान सीखे गए गुणांक देख सकते हैं। ये गुणांक उस सर्वश्रेष्ठ फिट लाइन के गुणांक को दर्शाते हैं, जो वास्तविक और अनुमानित चर के बीच कुल त्रुटि को न्यूनतम करता है।  
<br>

## 5. परीक्षण सेट पर भविष्यवाणियां करें

अब जब हमने एक मॉडल प्रशिक्षित कर लिया है, तो हम इसका उपयोग परीक्षण डेटासेट के लिए बीमारी की प्रगति y की भविष्यवाणी करने के लिए कर सकते हैं, [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) का उपयोग करके। इसका उपयोग डेटा समूहों के बीच रेखा खींचने के लिए किया जाएगा।  


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

वाह! 💃🕺 हमने अभी एक मॉडल को प्रशिक्षित किया और इसका उपयोग भविष्यवाणियां करने के लिए किया!

भविष्यवाणियां करते समय, tidymodels की परंपरा है कि हमेशा परिणामों का एक टिबल/डेटा फ्रेम तैयार किया जाए जिसमें मानकीकृत कॉलम नाम हों। यह मूल डेटा और भविष्यवाणियों को एक उपयोगी प्रारूप में संयोजित करना आसान बनाता है, जिसे बाद के कार्यों जैसे कि प्लॉटिंग के लिए उपयोग किया जा सकता है।

`dplyr::bind_cols()` कई डेटा फ्रेम्स को कॉलम के रूप में कुशलतापूर्वक जोड़ता है।


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. मॉडलिंग परिणामों का प्लॉट करें

अब, इसे विज़ुअली देखने का समय है 📈। हम टेस्ट सेट के सभी `y` और `bmi` मानों का एक स्कैटर प्लॉट बनाएंगे, फिर भविष्यवाणियों का उपयोग करके मॉडल के डेटा समूहों के बीच सबसे उपयुक्त स्थान पर एक रेखा खींचेंगे।

R में ग्राफ बनाने के लिए कई सिस्टम हैं, लेकिन `ggplot2` सबसे सुंदर और सबसे बहुमुखी में से एक है। यह आपको **स्वतंत्र घटकों को मिलाकर** ग्राफ बनाने की अनुमति देता है।


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ यहाँ थोड़ा सोचें कि यहाँ क्या हो रहा है। एक सीधी रेखा कई छोटे डेटा बिंदुओं के बीच से गुजर रही है, लेकिन यह वास्तव में क्या कर रही है? क्या आप देख सकते हैं कि इस रेखा का उपयोग करके आप यह अनुमान कैसे लगा सकते हैं कि एक नया, अनदेखा डेटा बिंदु प्लॉट के y अक्ष के संबंध में कहाँ फिट होगा? इस मॉडल के व्यावहारिक उपयोग को शब्दों में व्यक्त करने की कोशिश करें।

बधाई हो, आपने अपना पहला लीनियर रिग्रेशन मॉडल बनाया, इसके साथ एक भविष्यवाणी की, और इसे एक प्लॉट में प्रदर्शित किया!



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।  
